In [ ]:
#import libraries


import numpy
import pandas
import sklearn.linear_model
import sklearn.ensemble
import matplotlib
import seaborn
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten

In [ ]:
#import data
train_data = pandas.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test_data = pandas.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
#check training data
train_data.head()

In [ ]:
#check test data
test_data.head()

In [ ]:
#train data info
train_data.describe()

In [ ]:
#check for missing values
train_data.isnull().sum()

Есть отрицательные значения в цене и количестве проданного товара, что является нелогичным. Предположим, что ошибка заключается в ошибочно поставленном минусе, и возьмем модуль числа.

In [ ]:
#take absolute number of price and quantity 
train_data['item_price'] = train_data['item_price'].abs()
train_data['item_cnt_day'] = train_data['item_cnt_day'].abs()

Сгрупируем продажи по месяцам, так как требуется прогноз на месячные продажи

In [ ]:
#Grouping item price and item count per month
train_data = train_data.groupby(['date_block_num','shop_id','item_id']).agg({'item_price':'last','item_cnt_day':'sum'}).reset_index()
#Changing the name of the item_cnt_day to item_cnt_month
train_data = train_data.rename(columns={'item_cnt_day':'item_cnt_month'})

In [ ]:
#check training data
train_data.head()

Теперь данные готовы для дальнейшей работы. В первую очередь визуализируем данные.

In [ ]:
train_data['item_id'].hist()

In [ ]:
#Calculating Correlation
correlation = train_data.corr()

#Plotting correlation
matplotlib.pyplot.figure(figsize=(12,12))
corr_heatmap = seaborn.heatmap(correlation,annot=True,cmap="YlOrRd")

Никакой значимой информации получено не было.

Теперь подготовим тестовые данные. В тренировочных данных имеется информация по продажам в первые 33 месяца. Добавим 34 месяц, на который будем предсказывать продажи. Также добавим информацию о цене продукта из тренировочных данных.

In [ ]:
#Adding date_block_num to the Test data
test_data['date_block_num']=34
test_data = test_data[['date_block_num','shop_id','item_id']]
#Adding the prices
item_price = dict(train_data.groupby('item_id')['item_price'].last().reset_index().values)
test_data['item_price'] = test_data.item_id.map(item_price)
#check test data
test_data.head()

Наблюдаем пропущенные значения. Заменим их средней ценой.

In [ ]:
#replacing missings
test_data['item_price'] = test_data['item_price'].fillna(test_data['item_price'].mean())
test_data['item_price']

Подготовим данные для моделирования.

In [ ]:
#data split
x_train = train_data.drop('item_cnt_month',axis=1)
y_train = train_data["item_cnt_month"]
x_test = test_data

#check data
x_test.head()

Построим линейную регрессию и выведем R2

In [ ]:
linear_model = sklearn.linear_model.LinearRegression()
linear_model.fit(x_train,y_train)
linear_prediction = linear_model.predict(x_test)
#print(linear_prediction)
R2 = linear_model.score(x_train,y_train)
print (R2)

Соревнование оценивается по RMSE. Посчитаем ошибку

In [ ]:
#RMSE for linear
e = y_train - linear_model.predict(x_train)
MSE = numpy.square(e).mean()
RMSE = numpy.sqrt(MSE)
print (RMSE)

Ужасный результат. Попробуем регрессию случайного леса.

In [ ]:
rr_model = sklearn.ensemble.RandomForestRegressor(n_estimators=50)
rr_model.fit(x_train,y_train)
rr_predict = rr_model.predict(x_test)

In [ ]:
R2 = rr_model.score(x_train,y_train)
print (R2)

In [ ]:
#RMSE for random forest
e = y_train - rr_model.predict(x_train)
MSE = numpy.square(e).mean()
RMSE = numpy.sqrt(MSE)
print (RMSE)

Результат стал значительно лучше. Визуализируем зависимую переменную и прогноз, полученный двумя методами. 

In [ ]:
matplotlib.pyplot.plot(y_train)
matplotlib.pyplot.plot(rr_model.predict(x_train))
matplotlib.pyplot.plot(linear_model.predict(x_train))

Попробуем также ридж и лассо регрессии.

In [ ]:
#Lasso
lasso_model = sklearn.linear_model.Lasso(alpha=1.0)
lasso_model.fit(x_train,y_train)
lasso_predict = lasso_model.predict(x_test)
print(lasso_predict)

#Ridge
r_model = sklearn.linear_model.Ridge(alpha=1.0)
r_model.fit(x_train,y_train)
r_predict = r_model.predict(x_test)
print(r_predict)

In [ ]:
#R2 for lasso
R2 = lasso_model.score(x_train,y_train)
print (R2)

#RMSE for lasso
e = y_train - lasso_model.predict(x_train)
MSE = numpy.square(e).mean()
RMSE = numpy.sqrt(MSE)
print (RMSE)

In [ ]:
#R2 for ridge
R2 = r_model.score(x_train,y_train)
print (R2)

#RMSE for ridge
e = y_train - r_model.predict(x_train)
MSE = numpy.square(e).mean()
RMSE = numpy.sqrt(MSE)
print (RMSE)

Лассо и ридж не дали нормального результата. Будем использовать прогноз от регрессии случайного леса как финальный.

In [ ]:
rr_predict

In [ ]:
sample_submission= pandas.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
sample_submission.item_cnt_month=rr_predict
result=sample_submission
print(result)

result.to_csv("submission.csv",index=False)